In [1]:
from zipfile import ZipFile
zip = ZipFile('Resources/archive.zip')
zip.extractall('Resources')

In [2]:
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import findspark
from pyspark.sql import SparkSession


In [3]:
#Display all the columns (to see which to drop)
pd.set_option("display.max_columns", None)

## Read In CSV data

In [4]:
# Initialize spark
findspark.init()

# Create a SparkSession
#spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
spark = SparkSession.builder.appName('SparkSQL').config("spark.driver.memory", "2g").getOrCreate()

## Read In CSV data for ETF prices and ETF information


In [41]:
# Read in ETF data 
# ETF Prices
ETF_prices_path = 'Resources/ETF prices.csv'
ETF_prices= spark.read.csv(ETF_prices_path, header=True, inferSchema=True)

# ETF Prices
ETFs_path = 'Resources/ETFs.csv'
ETFs= spark.read.csv(ETFs_path, header=True, inferSchema=True)


In [51]:
# Read in ETF data to dataframes
# ETF Prices
ETF_prices_path = 'Resources/ETF prices.csv'
df_ETF_prices= pd.read_csv(
    Path(ETF_prices_path)
)

# ETF Prices
ETFs_path = 'Resources/ETFs.csv'
df_ETFs= pd.read_csv(
    Path(ETFs_path)
)


In [53]:
df_ETF_prices.head(5)

,fund_symbol,price_date,open,high,low,close,adj_close,volume
0,AAA,2020-09-09,25.10,25.12,25.07,25.07,24.85,17300
1,AAA,2020-09-10,25.06,25.07,25.05,25.07,24.85,23500
2,AAA,2020-09-11,25.04,25.05,25.02,25.03,24.81,33400
3,AAA,2020-09-14,25.01,25.06,25.01,25.02,24.80,13100
4,AAA,2020-09-15,25.02,25.03,25.01,25.01,24.79,12100


In [54]:
df_ETFs.head(5)

,fund_symbol,quote_type,region,fund_short_name,fund_long_name,currency,fund_category,fund_family,exchange_code,exchange_name,exchange_timezone,avg_vol_3month,avg_vol_10day,total_net_assets,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,inception_date,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,asset_stocks,asset_bonds,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,fund_price_cashflow_ratio,fund_price_earning_ratio,fund_price_sales_ratio,fund_bond_maturity,fund_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,fund_return_2008,category_return_2008,fund_return_2007,category_return_2007,fund_return_2006,category_return_2006,fund_return_2005,category_return_2005,fund_return_2004,category_return_2004,fund_return_2003,category_return_2003,fund_return_2002,category_return_2002,fund_return_2001,category_return_2001,fund_return_2000,category_return_2000,fund_alpha_3years,fund_beta_3years,fund_mean_annual_return_3years,fund_r_squared_3years,fund_stdev_3years,fund_sharpe_ratio_3years,fund_treynor_ratio_3years,fund_alpha_5years,fund_beta_5years,fund_mean_annual_return_5years,fund_r_squared_5years,fund_stdev_5years,fund_sharpe_ratio_5years,fund_treynor_ratio_5years,fund_alpha_10years,fund_beta_10years,fund_mean_annual_return_10years,fund_r_squared_10years,fund_stdev_10years,fund_sharpe_ratio_10years,fund_treynor_ratio_10years
0,AAAU,ETF,US,DWS RREEF Real Assets Fund - Cl,DWS RREEF Real Assets Fund - Class A,USD,NaN,DWS,PCX,NYSEArca,America/New_York,239238,255410.0,3.844486e+08,17.807,17.818,2.75,0.14146,19.44,-1.7200,-0.08848,16.69,1.03000,0.06171,The investment seeks total return in excess of...,NaN,2018-07-24,NaN,NaN,NaN,0.0018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Cayman Real Assets Fund Ltd."": 0.1098, CCI (""...",1.0000,2021-08-31,-0.0465,NaN,NaN,NaN,-0.0467,NaN,-0.0790,NaN,0.1460,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.23963,NaN,0.18579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.18,0.07,1.23,0.54,14.93,0.91,187.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AADR,ETF,US,AllianzGI Health Sciences Fund,Virtus AllianzGI Health Sciences Fund Class P,USD,Foreign Large Growth,Virtus,NGM,NasdaqGM,America/New_York,2596,3170.0,8.883616e+07,64.555,65.297,11.82,0.17042,69.36,-7.8645,-0.11339,57.54,3.95550,0.06874,The investment seeks long-term capital appreci...,0.0031,2010-07-20,NaN,Blend,Large,0.0110,0

In [57]:
ETF_prices.dtypes

[('fund_symbol', 'string'),
 ('price_date', 'date'),
 ('open', 'double'),
 ('high', 'double'),
 ('low', 'double'),
 ('close', 'double'),
 ('adj_close', 'double'),
 ('volume', 'bigint')]

In [120]:
ETFs.dtypes

[('fund_symbol', 'string'),
 ('quote_type', 'string'),
 ('region', 'string'),
 ('fund_short_name', 'string'),
 ('fund_long_name', 'string'),
 ('currency', 'string'),
 ('fund_category', 'string'),
 ('fund_family', 'string'),
 ('exchange_code', 'string'),
 ('exchange_name', 'string'),
 ('exchange_timezone', 'string'),
 ('avg_vol_3month', 'int'),
 ('avg_vol_10day', 'int'),
 ('total_net_assets', 'bigint'),
 ('day50_moving_average', 'double'),
 ('day200_moving_average', 'double'),
 ('week52_high_low_change', 'double'),
 ('week52_high_low_change_perc', 'double'),
 ('week52_high', 'double'),
 ('week52_high_change', 'double'),
 ('week52_high_change_perc', 'double'),
 ('week52_low', 'double'),
 ('week52_low_change', 'double'),
 ('week52_low_change_perc', 'double'),
 ('investment_strategy', 'string'),
 ('fund_yield', 'string'),
 ('inception_date', 'string'),
 ('annual_holdings_turnover', 'string'),
 ('investment_type', 'string'),
 ('size_type', 'string'),
 ('fund_annual_report_net_expense_ratio'

In [46]:
# Create temporary views for each ETF file/datafram
ETF_prices.createOrReplaceTempView('etfprices')
ETFs.createOrReplaceTempView('etfs')

In [47]:
spark.sql('SHOW COLUMNS in etfs').show()

+--------------------+
|            col_name|
+--------------------+
|         fund_symbol|
|          quote_type|
|              region|
|     fund_short_name|
|      fund_long_name|
|            currency|
|       fund_category|
|         fund_family|
|       exchange_code|
|       exchange_name|
|   exchange_timezone|
|      avg_vol_3month|
|       avg_vol_10day|
|    total_net_assets|
|day50_moving_average|
|day200_moving_ave...|
|week52_high_low_c...|
|week52_high_low_c...|
|         week52_high|
|  week52_high_change|
+--------------------+
only showing top 20 rows



In [48]:
spark.sql('SHOW COLUMNS in etfprices').show()

+-----------+
|   col_name|
+-----------+
|fund_symbol|
| price_date|
|       open|
|       high|
|        low|
|      close|
|  adj_close|
|     volume|
+-----------+



In [38]:
spark.sql('''
          DESCRIBE table etfs
          '''
).show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|         fund_symbol|   string|   NULL|
|          quote_type|   string|   NULL|
|              region|   string|   NULL|
|     fund_short_name|   string|   NULL|
|      fund_long_name|   string|   NULL|
|            currency|   string|   NULL|
|       fund_category|   string|   NULL|
|         fund_family|   string|   NULL|
|       exchange_code|   string|   NULL|
|       exchange_name|   string|   NULL|
|   exchange_timezone|   string|   NULL|
|      avg_vol_3month|   string|   NULL|
|       avg_vol_10day|   string|   NULL|
|    total_net_assets|   string|   NULL|
|day50_moving_average|   string|   NULL|
|day200_moving_ave...|   string|   NULL|
|week52_high_low_c...|   string|   NULL|
|week52_high_low_c...|   string|   NULL|
|         week52_high|   string|   NULL|
|  week52_high_change|   string|   NULL|
+--------------------+---------+-------+
only showing top

In [119]:
spark.sql('''
          DESCRIBE table etfprices
          '''
).show()

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|fund_symbol|   string|   NULL|
| price_date|     date|   NULL|
|       open|   double|   NULL|
|       high|   double|   NULL|
|        low|   double|   NULL|
|      close|   double|   NULL|
|  adj_close|   double|   NULL|
|     volume|   bigint|   NULL|
+-----------+---------+-------+



In [74]:
prices_column_info = pd.DataFrame(spark.catalog.listColumns('etfprices'))
prices_column_info

,name,description,dataType,nullable,isPartition,isBucket
0,fund_symbol,None,string,True,False,False
1,price_date,None,date,True,False,False
2,open,None,double,True,False,False
3,high,None,double,True,False,False
4,low,None,double,True,False,False
5,close,None,double,True,False,False
6,adj_close,None,double,True,False,False
7,volume,None,bigint,True,False,False


In [75]:
etfs_column_info = pd.DataFrame(spark.catalog.listColumns('etfs'))
etfs_column_info

,name,description,dataType,nullable,isPartition,isBucket
0,fund_symbol,None,string,True,False,False
1,quote_type,None,string,True,False,False
2,region,None,string,True,False,False
3,fund_short_name,None,string,True,False,False
4,fund_long_name,None,string,True,False,False
...,...,...,...,...,...,...
137,fund_mean_annual_return_10years,None,double,True,False,False
138,fund_r_squared_10years,None,double,True,False,False
139,fund_stdev_10years,None,double,True,False,False
140,fund_sharpe_ratio_10years,None,double,True,False,False


In [91]:
etfs_column_names = list(etfs_column_info['name'])
etfs_column_names[:10]

['fund_symbol',
 'quote_type',
 'region',
 'fund_short_name',
 'fund_long_name',
 'currency',
 'fund_category',
 'fund_family',
 'exchange_code',
 'exchange_name']

In [142]:
# grab all deleted column names from Mutual Fund data cleansing into lists
unnecessary = ['fund_short_name', 'fund_long_name','management_name', 'management_bio', '"investment_strategy']
one_count_drops = ['quote_type', 'region', 'currency', 'exchange_code', 'exchange_name', 'exchange_timezone']
high_NaN_drops = ['subsequent_investment', 'last_cap_gain', 'fund_max_front_end_sales_load', 
                  'fund_max_deferred_sales_load', 'fund_bond_maturity', 'fund_return_2008',
                  'fund_return_2007', 'fund_return_2006', 'fund_return_2005', 'fund_return_2004',
                  'fund_return_2003', 'fund_return_2002', 'fund_return_2001', 'fund_return_2000',
                  'fund_return_2021_q3', 'fund_return_2021_q2', 'fund_return_2008_q4', 'fund_return_2007_q4',
                  'fund_return_2007_q3', 'fund_return_2007_q2', 'fund_return_2007_q1', 'fund_return_2006_q4',
                  'fund_return_2006_q3','fund_return_2006_q2', 'fund_return_2006_q1', 'fund_return_2005_q4',
                  'fund_return_2005_q3', 'fund_return_2005_q2', 'fund_return_2005_q1', 'fund_return_2004_q4',
                  'fund_return_2004_q3', 'fund_return_2004_q2', 'fund_return_2004_q1', 'fund_return_2003_q4',
                  'fund_return_2003_q3','fund_return_2003_q2', 'fund_return_2003_q1', 'fund_return_2002_q4',
                  'fund_return_2002_q3', 'fund_return_2002_q2', 'fund_return_2002_q1', 'fund_return_2001_q4',
                  'fund_return_2001_q3', 'fund_return_2001_q2', 'fund_return_2001_q1', 'fund_return_2000_q4',
                  'fund_return_2000_q3', 'fund_return_2000_q2', 'fund_return_2000_q1']
qrtrly_calculable_drops = ['fund_return_2021_q1', 'fund_return_2020_q4','fund_return_2020_q3','fund_return_2020_q2',
                  'fund_return_2020_q1','fund_return_2019_q4','fund_return_2019_q3','fund_return_2019_q2',
                  'fund_return_2019_q1','fund_return_2018_q4','fund_return_2018_q3','fund_return_2018_q2',
                  'fund_return_2018_q1','fund_return_2017_q4','fund_return_2017_q3','fund_return_2017_q2',
                  'fund_return_2017_q1','fund_return_2016_q4','fund_return_2016_q3','fund_return_2016_q2',
                  'fund_return_2016_q1','fund_return_2015_q4','fund_return_2015_q3','fund_return_2015_q2',
                  'fund_return_2015_q1','fund_return_2014_q4','fund_return_2014_q3','fund_return_2014_q2',
                  'fund_return_2014_q1','fund_return_2013_q4','fund_return_2013_q3','fund_return_2013_q2',
                  'fund_return_2013_q1','fund_return_2012_q4','fund_return_2012_q3','fund_return_2012_q2',
                  'fund_return_2012_q1','fund_return_2011_q4','fund_return_2011_q3','fund_return_2011_q2',
                  'fund_return_2011_q1','fund_return_2010_q4','fund_return_2010_q3','fund_return_2010_q2',
                  'fund_return_2010_q1','fund_return_2009_q4','fund_return_2009_q3','fund_return_2009_q2',
                  'fund_return_2009_q1','fund_return_2008_q3','fund_return_2008_q2','fund_return_2008_q1',
                  'fund_alpha_3years','fund_beta_3years','fund_mean_annual_return_3years','fund_r_squared_3years',
                  'fund_stdev_3years','fund_sharpe_ratio_3years','fund_treynor_ratio_3years','fund_alpha_5years',
                  'fund_beta_5years','fund_mean_annual_return_5years','fund_r_squared_5years','fund_stdev_5years',
                  'fund_sharpe_ratio_5years','fund_treynor_ratio_5years','fund_alpha_10years','fund_beta_10years',
                  'fund_mean_annual_return_10years','fund_r_squared_10years','fund_stdev_10years','fund_sharpe_ratio_10years',
                  'fund_treynor_ratio_10years','fund_return_category_rank_ytd','fund_return_category_rank_1month',
                  'fund_return_category_rank_3months','fund_return_category_rank_1year','fund_return_category_rank_3years',
                  'fund_return_category_rank_5years','load_adj_return_1year','load_adj_return_3years','load_adj_return_5years',
                  'load_adj_return_10years','top10_holdings','esg_peer_count','peer_esg_min', 'peer_esg_avg', 'peer_esg_max',
                  'peer_environment_min', 'peer_environment_avg', 'peer_environment_max', 'peer_social_min', 'peer_social_avg',
                  'peer_social_max','peer_governance_min', 'peer_governance_avg', 'peer_governance_max']

In [80]:
len(one_count_drops)
len(high_NaN_drops)
len(qrtrly_calculable_drops)

97

In [143]:
cleanup_drop_list = unnecessary + one_count_drops + high_NaN_drops + qrtrly_calculable_drops
len(cleanup_drop_list)
cleanup_drop_list[:10]

['fund_short_name',
 'fund_long_name',
 'management_name',
 'management_bio',
 '"investment_strategy',
 'quote_type',
 'region',
 'currency',
 'exchange_code',
 'exchange_name']

In [144]:
etfs_clean_columns = []
for i in range(0, len(etfs_column_names)):
    if etfs_column_names[i] not in cleanup_drop_list:
        etfs_clean_columns.append(etfs_column_names[i])
len(etfs_clean_columns)

102

In [147]:
etfs_clean_columns

['fund_symbol',
 'fund_category',
 'fund_family',
 'avg_vol_3month',
 'avg_vol_10day',
 'total_net_assets',
 'day50_moving_average',
 'day200_moving_average',
 'week52_high_low_change',
 'week52_high_low_change_perc',
 'week52_high',
 'week52_high_change',
 'week52_high_change_perc',
 'week52_low',
 'week52_low_change',
 'week52_low_change_perc',
 'investment_strategy',
 'fund_yield',
 'inception_date',
 'annual_holdings_turnover',
 'investment_type',
 'size_type',
 'fund_annual_report_net_expense_ratio',
 'category_annual_report_net_expense_ratio',
 'asset_stocks',
 'asset_bonds',
 'fund_sector_basic_materials',
 'fund_sector_communication_services',
 'fund_sector_consumer_cyclical',
 'fund_sector_consumer_defensive',
 'fund_sector_energy',
 'fund_sector_financial_services',
 'fund_sector_healthcare',
 'fund_sector_industrials',
 'fund_sector_real_estate',
 'fund_sector_technology',
 'fund_sector_utilities',
 'fund_price_book_ratio',
 'fund_price_cashflow_ratio',
 'fund_price_earning_

In [115]:
# not working... switching to pandas

query = (f"select * from etfs where column_name IN (*{one_count_drops})")
spark.sql(query).show(10)

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'IN'.(line 1, pos 37)

== SQL ==
select * from etfs where column_name IN (*['quote_type', 'region', 'currency', 'exchange_code', 'exchange_name', 'exchange_timezone'])
-------------------------------------^^^


In [150]:
ETFs_clean_df = df_ETFs[etfs_clean_columns].copy()

In [155]:
ETFs_clean_df

,fund_symbol,fund_category,fund_family,avg_vol_3month,avg_vol_10day,total_net_assets,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,inception_date,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,asset_stocks,asset_bonds,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,fund_price_cashflow_ratio,fund_price_earning_ratio,fund_price_sales_ratio,fund_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings_total_assets,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,category_return_2008,category_return_2007,category_return_2006,category_return_2005,category_return_2004,category_return_2003,category_return_2002,category_return_2001,category_return_2000
0,AAAU,NaN,DWS,239238,255410.0,3.844486e+08,17.807,17.818,2.75,0.14146,19.44,-1.7200,-0.08848,16.69,1.03000,0.06171,The investment seeks total return in excess of...,NaN,2018-07-24,NaN,NaN,NaN,0.0018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,2021-08-31,-0.0465,NaN,NaN,NaN,-0.0467,NaN,-0.0790,NaN,0.1460,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.23963,NaN,0.18579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AADR,Foreign Large Growth,Virtus,2596,3170.0,8.883616e+07,64.555,65.297,11.82,0.17042,69.36,-7.8645,-0.11339,57.54,3.95550,0.06874,The investment seeks long-term capital appreci...,0.0031,2010-07-20,NaN,Blend,Large,0.0110,0.0066,NaN,NaN,0.2536,0.0736,0.1164,0.0000,0.0455,0.1125,0.0799,0.1394,0.0000,0.1791,0.0000,1.71,7.46,13.34,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3880,2021-06-30,0.0940,0.0410,0.0049,0.0161,0.0688,0.0508,0.2587,0.1047,0.0803,0.0233,0.1188,0.0824,0.0830,0.0243,7.0,3.0,0.12619,NaN,0.35885,NaN,-0.31185,NaN,0.46928,NaN,0.05575,NaN,0.04380,0.01427,-0.00830,-0.03065,0.20878,0.26387,0.12653,0.14598,-0.05806,-0.09042,NaN,0.11984,NaN,0.29080,-0.42427,0.31193,0.21884,NaN,NaN,NaN,NaN,NaN,NaN
2,AAXJ,Pacific/Asia ex-Japan Stk,American Century Investments,1036417,1287600.0,5.574672e+09,86.132,90.435,19.99,0.19514,102.44,-19.4300,-0.18967,82.45,0.56001,0.00679,The investment seeks the highest total return ...,0.0110,2008-08-13,0.23,Blend,Large,0.0070,0.0054,NaN,NaN,0.0523,0.1053,0.1683,0.0474,0.0266,0.1824,0.0524,0.0579,0.0391,0.2444,0.0239,2.03,11.70,18.00,1.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3005,2021-07-31,-0.0173,0.0984,-0.0709,-0.0199,-0.0600,0.0100,0.1859,0.0708,0.0804,0.0024,0.1067,0.0352,0.0535,0.0467,9.0,3.0,0.23865,NaN,0.17398,NaN,-0.14814,NaN,0.40517,NaN,0.04833,NaN,-0.09850,-0.08923,0.03925,0.02285,0.02478,0.02827,0.21386,0.20639,-0.20008,-0.20255,0.1553

In [159]:
# Looking at each column and the count of NaN in each column
NaN_list = []
for index, row in pd.DataFrame(ETFs_clean_df.isna().sum()).iterrows():
    NaN_list.append((index,row))

NaN_list

[('fund_symbol',
  0    0
  Name: fund_symbol, dtype: int64),
 ('fund_category',
  0    623
  Name: fund_category, dtype: int64),
 ('fund_family',
  0    0
  Name: fund_family, dtype: int64),
 ('avg_vol_3month',
  0    0
  Name: avg_vol_3month, dtype: int64),
 ('avg_vol_10day',
  0    15
  Name: avg_vol_10day, dtype: int64),
 ('total_net_assets',
  0    30
  Name: total_net_assets, dtype: int64),
 ('day50_moving_average',
  0    1
  Name: day50_moving_average, dtype: int64),
 ('day200_moving_average',
  0    1
  Name: day200_moving_average, dtype: int64),
 ('week52_high_low_change',
  0    1
  Name: week52_high_low_change, dtype: int64),
 ('week52_high_low_change_perc',
  0    1
  Name: week52_high_low_change_perc, dtype: int64),
 ('week52_high',
  0    1
  Name: week52_high, dtype: int64),
 ('week52_high_change',
  0    3
  Name: week52_high_change, dtype: int64),
 ('week52_high_change_perc',
  0    3
  Name: week52_high_change_perc, dtype: int64),
 ('week52_low',
  0    1
  Name: wee

In [160]:
# Find columns with only 1 value to drop
ETFs_clean_df.loc[: , ETFs_clean_df.dtypes== "object"].nunique()

fund_symbol            2310
fund_category            83
fund_family             150
investment_strategy     791
inception_date         1097
investment_type           3
size_type                 3
returns_as_of_date        9
dtype: int64